In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'F:\ImpactHistology\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'F:\polarimetry\calibration'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.39s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [ ]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    for tissue_type in tissue_types:
        
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_list = selection_of_ROIs.square_selection(path_folder_temp, path_folder, 
                                                             WM, tissue_type, path_align_folder)
        
        # 3. actually do the alignment
        propagation_list = automatic_ROI_propagation.generate_combined_mask(propagation_list, 
                                                                            path_align_folder)
        automatic_ROI_propagation.do_alignment(path_align_folder)
        output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
        
        # 4. and collect the data after propagation
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_list, output_folders)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [5]:
from freezingeffect import parameter_comparaison

In [6]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_3',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_4',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_8',
 'F:\\Imp

#### 4.2. Get the data for the measurement

In [7]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,102.287019,30.135654,117.653631,102.287019,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.763450,0.023632,0.760294,0.763462,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.036605,0.012471,0.030303,0.035345,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,9.230560,4.581735,8.044693,8.433148,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,74.421430,49.618661,57.318436,74.421430,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.848344,0.038575,0.841176,0.847996,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.055518,0.013299,0.055556,0.054897,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,10.758224,6.289547,6.536313,9.838684,None


In [8]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [9]:
data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,102.287019,30.135654,117.653631,102.287019,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.763450,0.023632,0.760294,0.763462,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.036605,0.012471,0.030303,0.035345,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,9.230560,4.581735,8.044693,8.433148,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,74.421430,49.618661,57.318436,74.421430,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.848344,0.038575,0.841176,0.847996,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.055518,0.013299,0.055556,0.054897,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,10.758224,6.289547,6.536313,9.838684,None


In [10]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [11]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,12.656324,13.967605,12.614383,8.843303,13.847392,16.725589,13.207905,35.585788,14.925649,17.288618,...,17.794185,12.191899,12.365931,13.600335,27.837086,14.139676,24.775550,23.260515,13.751354,13.461304
1,12.819288,NaN,9.111501,12.825967,NaN,12.675664,13.926056,20.625196,12.780575,11.867152,...,15.311772,15.723257,12.912673,14.534859,13.434312,19.158055,11.985926,13.042563,11.916742,15.838989


In [12]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.000000,1.0,1.00000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
1,1.012876,NaN,0.72231,1.450359,NaN,0.757861,1.054373,0.579591,0.856283,0.686414,...,0.860493,1.289648,1.044214,1.068713,0.482605,1.354915,0.48378,0.560717,0.866587,1.176631


#### 4.3. Statistical analysis

In [13]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [14]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value   n_1   n_2  before/GM  \
parameter      data_type time                                                
azimuth        GM        T0    3000.0  7.985458e-23  50.0  60.0  29.289259   
                         T1    1250.0  1.000000e+00  50.0  50.0  29.289259   
               WM        T0    3300.0  2.054268e-23  55.0  60.0  28.783840   
                         T1    1512.5  1.000000e+00  55.0  55.0  28.783840   
depolarization GM        T0    1200.0  4.949484e-02  50.0  60.0   1.004693   
                         T1    1250.0  1.000000e+00  50.0  50.0   1.004693   
               WM        T0    2160.0  2.072315e-03  55.0  60.0   1.005329   
                         T1    1512.5  1.000000e+00  55.0  55.0   1.005329   
retardance     GM        T0    1260.0  1.162320e-01  50.0  60.0   0.933460   
                         T1    1250.0  1.000000e+00  50.0  50.0   0.933460   
               WM        T0    1500.0  3.661803e-01  55.0  60.0   0.956134   
                         T1    1512.5  1.000000e+00  55.0  55.0   0.956134   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        22.001284   0.000000      0.000000  
                         T1        22.001284  29.289259     22.001284  
               WM        T0        25.187281   0.000000      0.000000  
                         T1        25.187281  28.783840     25.187281  
depolarization GM        T0         0.047336   1.000000      0.000000  
                         T1         0.047336   1.004693      0.047336  
               WM        T0         0.014719   1.000000      0.000000  
                         T1         0.014719   1.005329      0.014719  
retardance     GM        T0         0.404266   1.000000      0.000000  
                         T1         0.404266   0.933460      0.404266  
               WM        T0         0.252183   1.000000      0.000000  
                         T1         0.252183   0.956134      0.252183

In [15]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [16]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.366346,1.046583,1.166668,NaN,1.289905,0.909171,1.920179,0.691595,NaN,1.105349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))